# Generating images with MX-Font model from a reference style
In this example we'll generate images with trained LF-Font model.

### 1. Loading packages
* First, load the packages used in this code.
* All of the packages are avilable in `pip`.

In [1]:
import json
from pathlib import Path
from PIL import Image
from itertools import chain

import torch
from sconf import Config
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

* These modules are defined in this repository.

In [2]:
from base.dataset import read_font, render
from base.utils import save_tensor_to_image, load_reference
from LF.models import Generator
from inference import infer_LF

### 2. Build model
* Build and load the trained model.
* `weight_path` : 
    * The location of the trained model weight.
* `emb_dim` :
    * The dimension of embedding blocks in the trained model weight.
* `decomposition` :
    * The location of the pre-defined decomposition rule file.
* `primals` :
    * The location of the primals list file.
    * The order of primals list should be identical to that used for the training.

In [3]:
###############################################################
weight_path = "temp/outputs2/checkpoints/last.pth"  # path to weight to infer
emb_dim = 8
decomposition = "data/kor/decomposition.json"
primals = "data/kor/primals.json"
###############################################################

decomposition = json.load(open(decomposition))
primals = json.load(open(primals))
n_comps = len(primals)

def decompose_to_ids(char):
    dec = decomposition[char]
    comp_ids = [primals.index(d) for d in dec]
    return comp_ids

cfg = Config("cfgs/LF/p2/default.yaml")

gen = Generator(n_comps=n_comps, emb_dim=emb_dim).cuda().eval()
weight = torch.load(weight_path)
if "generator_ema" in weight:
    weight = weight["generator_ema"]
gen.load_state_dict(weight)

<All keys matched successfully>

In [4]:
weight.keys()

odict_keys(['comp_enc.layers.0.conv.weight', 'comp_enc.layers.0.conv.bias', 'comp_enc.layers.1.conv.weight', 'comp_enc.layers.1.conv.bias', 'comp_enc.layers.2.gc.k_proj.weight', 'comp_enc.layers.2.gc.k_proj.bias', 'comp_enc.layers.2.gc.transform.0.weight', 'comp_enc.layers.2.gc.transform.0.bias', 'comp_enc.layers.2.gc.transform.1.weight', 'comp_enc.layers.2.gc.transform.1.bias', 'comp_enc.layers.2.gc.transform.3.weight', 'comp_enc.layers.2.gc.transform.3.bias', 'comp_enc.layers.3.conv.weight', 'comp_enc.layers.3.conv.bias', 'comp_enc.layers.4.ChannelGate.mlp.1.weight', 'comp_enc.layers.4.ChannelGate.mlp.1.bias', 'comp_enc.layers.4.ChannelGate.mlp.3.weight', 'comp_enc.layers.4.ChannelGate.mlp.3.bias', 'comp_enc.layers.4.SpatialGate.spatial.conv.weight', 'comp_enc.layers.4.SpatialGate.spatial.bn.weight', 'comp_enc.layers.4.SpatialGate.spatial.bn.bias', 'comp_enc.layers.4.SpatialGate.spatial.bn.running_mean', 'comp_enc.layers.4.SpatialGate.spatial.bn.running_var', 'comp_enc.layers.4.Spati

### 3. Load reference images.
* `ref_path`: 
    * The path of reference font or images.
    * If you are using a ttf file, set this to the location of the ttf file.
    * If you want to use rendered images, set this to the path to the directory which contains the reference images.
* `ref_chars`:
    * The characters of reference images.
    * If this is `None`, all the available images will be loaded.
* `extension`:
    * If you are using ttf files, set this to "ttf".
    * If you are using image files, set this to their extension(png, jpg, etc..).

In [5]:
########################################################
ref_path = "data_example/kor/png"
extension = "png"
ref_chars = "값넋닻볘츄퀭핥훟"
## Comment upper lines and uncomment lower lines to test with ttf files.
# extension = "ttf"
# ref_chars = "값같곬곶깎넋늪닫닭닻됩뗌략몃밟볘뺐뽈솩쐐앉않얘얾엌옳읊죡쮜춰츄퀭틔핀핥훟"
########################################################

ref_dict, load_img = load_reference(ref_path, extension, ref_chars)

In [7]:
ref_dict

{'UhBee_charming': ['값', '넋', '닻', '볘', '츄', '퀭', '핥', '훟'],
 'UhBee_HanByeol': ['값', '넋', '닻', '볘', '츄', '퀭', '핥', '훟']}

<function base.utils.load.load_reference.<locals>.load_img(key, char)>

### 4. Generate the images.
* `gen_chars`: The characters to generate.
* `save_dir`: Path to save the generated images.
* `source_path`: Path to the source.
* `source_ext`: Extension of the source file. If you are using image files, set this to the image's extension.
* `batch_size`: The number of images inferred at once.

In [6]:
########################################################
gen_chars = "좋은하루되세요"  # Characters to generate
save_dir = "./result/lf"  # Directory where you want to save generated images
source_path = "data/kor/source.ttf"
source_ext = "ttf"
batch_size = 16
########################################################

infer_LF(gen, save_dir, source_path, source_ext, gen_chars, ref_dict, load_img,
         decomposition, primals, batch_size)

{}

In [14]:
gen_chars, ref_dict, load_img,decomposition, primals, batch_size

('좋은하루되세요',
 {'UhBee_charming': ['값', '넋', '닻', '볘', '츄', '퀭', '핥', '훟'],
  'UhBee_HanByeol': ['값', '넋', '닻', '볘', '츄', '퀭', '핥', '훟']},
 <function base.utils.load.load_reference.<locals>.load_img(key, char)>,
 {'가': ['ㄱ', 'ㅏ'],
  '각': ['ㄱ', 'ㅏ', 'ㄱ'],
  '갂': ['ㄱ', 'ㅏ', 'ㄱ', 'ㄱ'],
  '갃': ['ㄱ', 'ㅏ', 'ㄱ', 'ㅅ'],
  '간': ['ㄱ', 'ㅏ', 'ㄴ'],
  '갅': ['ㄱ', 'ㅏ', 'ㄴ', 'ㅈ'],
  '갆': ['ㄱ', 'ㅏ', 'ㄴ', 'ㅎ'],
  '갇': ['ㄱ', 'ㅏ', 'ㄷ'],
  '갈': ['ㄱ', 'ㅏ', 'ㄹ'],
  '갉': ['ㄱ', 'ㅏ', 'ㄹ', 'ㄱ'],
  '갊': ['ㄱ', 'ㅏ', 'ㄹ', 'ㅁ'],
  '갋': ['ㄱ', 'ㅏ', 'ㄹ', 'ㅂ'],
  '갌': ['ㄱ', 'ㅏ', 'ㄹ', 'ㅅ'],
  '갍': ['ㄱ', 'ㅏ', 'ㄹ', 'ㅌ'],
  '갎': ['ㄱ', 'ㅏ', 'ㄹ', 'ㅍ'],
  '갏': ['ㄱ', 'ㅏ', 'ㄹ', 'ㅎ'],
  '감': ['ㄱ', 'ㅏ', 'ㅁ'],
  '갑': ['ㄱ', 'ㅏ', 'ㅂ'],
  '값': ['ㄱ', 'ㅏ', 'ㅂ', 'ㅅ'],
  '갓': ['ㄱ', 'ㅏ', 'ㅅ'],
  '갔': ['ㄱ', 'ㅏ', 'ㅅ', 'ㅅ'],
  '강': ['ㄱ', 'ㅏ', 'ㅇ'],
  '갖': ['ㄱ', 'ㅏ', 'ㅈ'],
  '갗': ['ㄱ', 'ㅏ', 'ㅊ'],
  '갘': ['ㄱ', 'ㅏ', 'ㅋ'],
  '같': ['ㄱ', 'ㅏ', 'ㅌ'],
  '갚': ['ㄱ', 'ㅏ', 'ㅍ'],
  '갛': ['ㄱ', 'ㅏ', 'ㅎ'],
  '개': ['ㄱ', 'ㅏ', 'ㅣ'],
  '객': ['ㄱ', 'ㅏ', 'ㅣ', 'ㄱ'],
  '갞'